## Passing arguments to threads

In [ ]:
def task(num):
    print(f"I'm the task {num}")

for i in range(10):
    t = threading.Thread(target=task, args=(i,))
    t.start()

## [Thread synchronization](https://pymotw.com/3/threading/index.html#signaling-between-threads)

In [3]:
import logging
import threading

def wait_for_an_event(event):
    print("I'm doing something")
    logging.debug("waitting for an event from other thread ...")
    e = event.wait()
    logging.debug(f"received event {e}")

def wait_for_an_event_with_timeout(event, timeout):
    while not event.is_set():
        print("I'm also doing something")
        logging.debug('waitting for an event from other tread, but with a timeout ...')
        e = event.wait(timeout)
        logging.debug(f"received event {e}")

logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

e = threading.Event()
threading.Thread(target=wait_for_an_event, args=(e,),).start()
threading.Thread(target=wait_for_an_event_with_timeout, args=(e, 2),).start()

logging.debug('waiting before calling Event.set()')
e.set()
logging.debug('event is set')

(Thread-6  ) waitting for an event from other thread ...
(Thread-7  ) waitting for an event from other tread, but with a timeout ...
(MainThread) waiting before calling Event.set()
(MainThread) event is set


I'm doing something
I'm also doing something


(Thread-6  ) received event True
(Thread-7  ) received event True


## 4. Synchronizing parallel tasks

### 4.1. Using a queue

With threads:

With processes:

In [ ]:
import multiprocessing
import time
import random

class Producer():
    def __init__(self, q, iters=10):
        super(Producer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.full():
                item = random.randint(1,10)
                self.q.put(item)
                print('Produced {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return
    
    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

class Consumer():
    def __init__(self, q, iters=10):
        super(Consumer,self).__init__()
        self.iters = iters
        self.q = q

    def run(self):
        i = 0
        while i < self.iters:
            if not self.q.empty():
                item = self.q.get()
                print('Consumed {} (queue length = {})'.format(item, self.q.qsize()))
                i += 1
                time.sleep(random.random())
        return

    def start(self):
        process = multiprocessing.Process(target=self.run)
        process.start()
        return process

queue_length = 10
q = multiprocessing.Queue(queue_length)

p = Producer(q)
task_p = p.start()
c = Consumer(q)
task_c = c.start()

task_p.join()
task_c.join()

print('done')